## Finetune

我analysis.py对脚本实现了对于一个subject的adaption。具体流程是finetune on condition 1的数据，然后test on 其他condition。

这里面有一些参数

subject = "S001"
config.runner.weight_min = 0.5
config.runner.weight_max = 0.5

subject = 就是我们想finetune哪个subject。weight_min和weight_max决定了在train的时候不同值对loss的影响。

现在，能不能帮我写个新的脚本，把这个analysis拓展到所有subject。具体来说，对于每个subject，
- 用weight_min = 1e-6, 0.1, 0.5, 0.9, 1-1e-6, weight_min = 1-1e-6, 0.9, 0.5, 0.1, 1e-6, 这5种parameter来finetune，所以我们会得到5个model
- 用每个model来predict这个subject的condition != 1的数据，那我们会得到5组（PredMinBP，PredMaxBP），加上（TrueMinBP，TrueMaxBP）的groud truth，对于每个subject我们就会有6组数，12个column
对于每个subject，请重复这个操作。注意，每个subject是独立train的，需要每次都重新load ckpt然后finetune。每个subject的每个parameter组合也是独立train的。

还有，注意虽然我说对所有subject做，但是其实只要对split=1或者2的subject来做就行，我的profile文件里有个column split。我只关心里面split=1或者2的subject。虽然这个split assign给了每个sample，但是其实是每个subject的所有sample都会被放进其中一个set。

在对所有split属于1，2的subject都做完，我们会得到一个N*(K+2*6) 的profile文件，其中K是原本profile文件里data.profile.copy()的column，是每个subject的Metadata。

如果有的subject没有condition=1的数据，或者没有condition！=1的数据，那就跳过这个subject，最终的profile文件里面也不用包含这个subject的samples。

请把最终的这个profile文件保存到data/presentation/ResultFintune这个文件夹里，记得新建一下这个文件夹。

如果可以，请把这个文件写在script/temp.py。我可以自己来跑这个文件。这个文件在跑的时候应该有三层tqdm bar，第一层展示目前subject进度，第二层展示这个subject进行到哪个parameters set了，第三层展示本次train的进度。

In [ ]:
from __future__ import annotations

import os
from pathlib import Path
import warnings

import lightning
import numpy as np
import pandas as pd
import torch
import tqdm

import src
import config

# Which config to use
config_name = "Finetune"


def _device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    if torch.backends.mps.is_available():
        return "mps"
    return "cpu"


def _set_seed_and_precision() -> None:
    torch.set_float32_matmul_precision("medium")
    lightning.seed_everything(42, workers=True, verbose=False)
    warnings.filterwarnings("ignore", message=".*MPS.*fallback.*")


def _load_model(cfg: config.Config, device: str) -> src.Model:
    if cfg.trainer.ckpt_load_path is None:
        raise ValueError("config.trainer.ckpt_load_path must be set")
    model = src.Model(**vars(cfg.model))
    src.Trainer.ckptLoader_(model, cfg.trainer.ckpt_load_path).to(device)
    model.freeze()
    return model


def _loss_shape(x: torch.Tensor, y: torch.Tensor, cfg) -> torch.Tensor:
    return torch.nn.functional.smooth_l1_loss(  # (B,)
        input=torch.stack([
            torch.roll(x, shifts=s, dims=-1)
            for s in range(-cfg.runner.K, cfg.runner.K + 1)
        ], dim=1)[:, :, cfg.runner.K:-cfg.runner.K],
        target=y.unsqueeze(1).expand(
            (-1, 2 * cfg.runner.K + 1, -1)
        )[:, :, cfg.runner.K:-cfg.runner.K],
        reduction="none",
    ).mean(dim=-1).min(dim=-1).values.mean()


def _loss_min(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.smooth_l1_loss(
        x.min(dim=-1).values, y.min(dim=-1).values
    )


def _loss_max(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.smooth_l1_loss(
        x.max(dim=-1).values, y.max(dim=-1).values
    )


def _weight_tag(w_min: float, w_max: float) -> str:
    def fmt(v: float) -> str:
        txt = f"{v:.6f}"
        txt = txt.rstrip("0").rstrip(".") if "." in txt else txt
        return txt.replace(".", "p")

    return f"wmin{fmt(w_min)}_wmax{fmt(w_max)}"


def main() -> None:
    _set_seed_and_precision()
    device = _device()

    cfg: config.Config = getattr(cfg, config_name)()
    data = src.DataModule(**vars(cfg.data))
    data.setup()

    profile = data.profile.copy()
    split_mask = profile["split"].isin([1, 2])
    subjects = sorted(profile.loc[split_mask, "subject"].unique())

    weight_pairs: list[tuple[float, float]] = [
        (1e-6, 1 - 1e-6),
        (0.1, 0.9),
        (0.5, 0.5),
        (0.9, 0.1),
        (1 - 1e-6, 1e-6),
    ]

    out_rows: list[pd.DataFrame] = []
    subjects_bar = tqdm.tqdm(subjects, desc="Subjects", position=0)
    for subject in subjects_bar:
        subjects_bar.set_postfix_str(subject)
        subject_profile = profile.loc[profile["subject"] == subject]
        cond_eq1_mask = subject_profile["condition"] == 1
        cond_ne1_mask = subject_profile["condition"] != 1
        if (not cond_eq1_mask.any()) or (not cond_ne1_mask.any()):
            subjects_bar.write(
                f"Skip {subject}: missing condition 1 or condition != 1 data"
            )
            continue

        base_out = subject_profile.loc[cond_ne1_mask].copy().reset_index(drop=True)
        true_filled = False

        weight_bar = tqdm.tqdm(
            weight_pairs, desc="Weight sets", position=1, leave=False
        )
        for w_min, w_max in weight_bar:
            weight_bar.set_postfix({"w_min": w_min, "w_max": w_max})
            cfg.objective.weight_min = w_min
            cfg.objective.weight_max = w_max

            model = _load_model(cfg, device)

            # backbone outputs for finetuning
            x_train, y_train = [], []
            for batch in data.train_dataloader(subject=subject):
                batch = [b.to(device) for b in batch]
                with torch.no_grad():
                    x_train.append(model.forward(batch[0], batch[1], pool_dim=1))
                y_train.append(batch[2])
            x_train = torch.cat(x_train, dim=0)
            y_train = torch.cat(y_train, dim=0)

            x_valid, y_valid = [], []
            for batch in data.val_dataloader(subject=subject):
                batch = [b.to(device) for b in batch]
                with torch.no_grad():
                    x_valid.append(model.forward(batch[0], batch[1], pool_dim=1))
                y_valid.append(batch[2])
            x_valid = torch.cat(x_valid, dim=0)
            y_valid = torch.cat(y_valid, dim=0)

            optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.objective.lr)

            epoch_bar = tqdm.trange(
                cfg.trainer.max_epochs, desc="Epochs", position=2, leave=False
            )
            for _ in epoch_bar:
                model.train()
                x = model.forwardAdapter(x_train)
                train_loss_shape = _loss_shape(x, y_train, cfg)
                train_loss_min = _loss_min(x, y_train)
                train_loss_max = _loss_max(x, y_train)
                train_loss = (
                    cfg.objective.weight_shape * train_loss_shape
                    + cfg.objective.weight_min * train_loss_min
                    + cfg.objective.weight_max * train_loss_max
                )
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()

                model.eval()
                with torch.no_grad():
                    x_val = model.forwardAdapter(x_valid)
                    valid_loss_shape = _loss_shape(x_val, y_valid, cfg)
                    valid_loss_min = _loss_min(x_val, y_valid)
                    valid_loss_max = _loss_max(x_val, y_valid)
                    valid_loss = (
                        cfg.objective.weight_shape * valid_loss_shape
                        + cfg.objective.weight_min * valid_loss_min
                        + cfg.objective.weight_max * valid_loss_max
                    )
                epoch_bar.set_postfix(
                    {
                        "train": float(train_loss),
                        "valid": float(valid_loss),
                    }
                )

            # prediction on this subject (all conditions), then keep condition != 1
            model.eval()
            result_batches = []
            for batch in data.test_dataloader(subject=subject):
                x, channel_idx, y = batch
                x, channel_idx, y = x.to(device), channel_idx.to(device), y.to(device)
                with torch.no_grad():
                    x_pred = model.forwardRegression(
                        x, channel_idx, adapter=True
                    )
                result_batches.append(torch.cat([
                    x.detach().cpu(),                       # (B, 4, T)
                    y.detach().cpu().unsqueeze(1),          # (B, 1, T)
                    x_pred.detach().cpu().unsqueeze(1),     # (B, 1, T)
                ], dim=1))
            result = torch.cat(result_batches, dim=0)       # (N, 5, T)
            result = torch.cat([
                result,
                data.denormalize(result[:, 3, :]).unsqueeze(1),
                data.denormalize(result[:, 4, :]).unsqueeze(1),
            ], dim=1).detach().cpu()                        # (N, 7, T)

            true_min = result[:, 5].min(dim=1).values.numpy()
            true_max = result[:, 5].max(dim=1).values.numpy()
            pred_min = result[:, 6].min(dim=1).values.numpy()
            pred_max = result[:, 6].max(dim=1).values.numpy()

            cond_ne1_array = cond_ne1_mask.to_numpy()
            if not true_filled:
                base_out["TrueMinBP"] = true_min[cond_ne1_array]
                base_out["TrueMaxBP"] = true_max[cond_ne1_array]
                true_filled = True

            tag = _weight_tag(w_min, w_max)
            base_out[f"PredMinBP_{tag}"] = pred_min[cond_ne1_array]
            base_out[f"PredMaxBP_{tag}"] = pred_max[cond_ne1_array]

            if device == "cuda":
                torch.cuda.empty_cache()

        out_rows.append(base_out)

    if not out_rows:
        print("No subjects processed; nothing saved.")
        return

    out_df = pd.concat(out_rows, axis=0).reset_index(drop=True)
    out_dir = Path("data/presentation/ResultFintune")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / "profile.csv"
    out_df.to_csv(out_path, index=False)
    print(f"Saved aggregated profile to {out_path} (rows={len(out_df)})")

# Ensure the working directory is the repo root so relative paths resolve
os.chdir(Path(__file__).resolve().parents[1])
main()

## Analysis

现在，趁着代码在跑，帮我写一个能分析这个profile的脚本。

我现在对于最终整体的MAE是多少非常感兴趣。那有两种算法。第一种算法是所有sample用同一个parameter set的情况下，最终的MAE。注意，min和max分别用不同parameter set：一个parameter set让min的MAE最低，一个parameter set让max的MAE最低。我们要求的只是同一个parameter set access  subject/samples。

请分别告诉我min和max的哪个parameter set能达到最好的MAE，最好的MAE是多少？然后请visualize一下per subject的MAE。然后visualize一下所有sample画在的，横坐标是prediction，纵坐标是group truth。可以color by subject。


第二种算法是，每个subject的min和max我们都用最好的那个parameter set的prediction value当作最终的。剩下的analysis和第一种算法一样，也告诉我最好的MAE多少，visualize一下per subject MAE，还有visulize一下所有sample画在一起的，横坐标是prediction BP，纵坐标是group truth。可以color by subject。注意visulize的时候min和max要分开。

请写在script/temp.ipynb里面，方便我看所有的图。

In [ ]:
from __future__ import annotations

import dataclasses
import os
from pathlib import Path
import warnings

import lightning
import numpy as np
import pandas as pd
import torch
import tqdm

import src

# Which config to use
config_name = "Finetune"


def _device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    if torch.backends.mps.is_available():
        return "mps"
    return "cpu"


def _set_seed_and_precision() -> None:
    torch.set_float32_matmul_precision("medium")
    lightning.seed_everything(42, workers=True, verbose=False)
    warnings.filterwarnings("ignore", message=".*MPS.*fallback.*")


def _load_model(cfg: config.Config, device: str) -> src.Model:
    if cfg.trainer.ckpt_load_path is None:
        raise ValueError("config.trainer.ckpt_load_path must be set")
    model = src.Model(**vars(cfg.model))
    src.Trainer.ckptLoader_(model, cfg.trainer.ckpt_load_path).to(device)
    model.freeze()
    return model


def _loss_shape(x: torch.Tensor, y: torch.Tensor, cfg) -> torch.Tensor:
    return torch.nn.functional.smooth_l1_loss(  # (B,)
        input=torch.stack([
            torch.roll(x, shifts=s, dims=-1)
            for s in range(-cfg.runner.K, cfg.runner.K + 1)
        ], dim=1)[:, :, cfg.runner.K:-cfg.runner.K],
        target=y.unsqueeze(1).expand(
            (-1, 2 * cfg.runner.K + 1, -1)
        )[:, :, cfg.runner.K:-cfg.runner.K],
        reduction="none",
    ).mean(dim=-1).min(dim=-1).values.mean()


def _loss_min(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.smooth_l1_loss(
        x.min(dim=-1).values, y.min(dim=-1).values
    )


def _loss_max(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.smooth_l1_loss(
        x.max(dim=-1).values, y.max(dim=-1).values
    )


def _weight_tag(w_min: float, w_max: float) -> str:
    def fmt(v: float) -> str:
        txt = f"{v:.6f}"
        txt = txt.rstrip("0").rstrip(".") if "." in txt else txt
        return txt.replace(".", "p")

    return f"wmin{fmt(w_min)}_wmax{fmt(w_max)}"


def main() -> None:
    _set_seed_and_precision()
    device = _device()

    config: src.config.Config = getattr(src.config, config_name)()
    data = src.data.DataModule(**vars(config.data))
    data.setup()

    profile = data.profile.copy()
    split_mask = profile["split"].isin([1, 2])
    subjects = sorted(profile.loc[split_mask, "subject"].unique())

    weight_pairs: list[tuple[float, float]] = [
        (1e-6, 1 - 1e-6),
        (0.1, 0.9),
        (0.5, 0.5),
        (0.9, 0.1),
        (1 - 1e-6, 1e-6),
    ]

    out_rows: list[pd.DataFrame] = []
    subjects_bar = tqdm.tqdm(subjects, desc="Subjects", position=0)
    for subject in subjects_bar:
        subjects_bar.set_postfix_str(subject)
        subject_profile = profile.loc[profile["subject"] == subject]
        cond_eq1_mask = subject_profile["condition"] == 1
        cond_ne1_mask = subject_profile["condition"] != 1
        if (not cond_eq1_mask.any()) or (not cond_ne1_mask.any()):
            subjects_bar.write(
                f"Skip {subject}: missing condition 1 or condition != 1 data"
            )
            continue

        base_out = subject_profile.loc[cond_ne1_mask].copy().reset_index(drop=True)
        true_filled = False

        weight_bar = tqdm.tqdm(
            weight_pairs, desc="Weight sets", position=1, leave=False
        )
        for w_min, w_max in weight_bar:
            weight_bar.set_postfix({"w_min": w_min, "w_max": w_max})
            config.objective.weight_min = w_min
            config.objective.weight_max = w_max

            model = _load_model(config, device)

            # backbone outputs for finetuning
            x_train, y_train = [], []
            for batch in data.train_dataloader(subject=subject):
                batch = [b.to(device) for b in batch]
                with torch.no_grad():
                    x_train.append(model.forward(batch[0], batch[1], pool_dim=1))
                y_train.append(batch[2])
            x_train = torch.cat(x_train, dim=0)
            y_train = torch.cat(y_train, dim=0)

            x_valid, y_valid = [], []
            for batch in data.val_dataloader(subject=subject):
                batch = [b.to(device) for b in batch]
                with torch.no_grad():
                    x_valid.append(model.forward(batch[0], batch[1], pool_dim=1))
                y_valid.append(batch[2])
            x_valid = torch.cat(x_valid, dim=0)
            y_valid = torch.cat(y_valid, dim=0)

            optimizer = torch.optim.AdamW(model.parameters(), lr=config.objective.lr)

            epoch_bar = tqdm.trange(
                config.trainer.max_epochs, desc="Epochs", position=2, leave=False
            )
            for _ in epoch_bar:
                model.train()
                x = model.forwardAdapter(x_train)
                train_loss_shape = _loss_shape(x, y_train, config)
                train_loss_min = _loss_min(x, y_train)
                train_loss_max = _loss_max(x, y_train)
                train_loss = (
                    config.objective.weight_shape * train_loss_shape
                    + config.objective.weight_min * train_loss_min
                    + config.objective.weight_max * train_loss_max
                )
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()

                model.eval()
                with torch.no_grad():
                    x_val = model.forwardAdapter(x_valid)
                    valid_loss_shape = _loss_shape(x_val, y_valid, config)
                    valid_loss_min = _loss_min(x_val, y_valid)
                    valid_loss_max = _loss_max(x_val, y_valid)
                    valid_loss = (
                        config.objective.weight_shape * valid_loss_shape
                        + config.objective.weight_min * valid_loss_min
                        + config.objective.weight_max * valid_loss_max
                    )
                epoch_bar.set_postfix(
                    {
                        "train": float(train_loss),
                        "valid": float(valid_loss),
                    }
                )

            # prediction on this subject (all conditions), then keep condition != 1
            model.eval()
            result_batches = []
            for batch in data.test_dataloader(subject=subject):
                x, channel_idx, y = batch
                x, channel_idx, y = x.to(device), channel_idx.to(device), y.to(device)
                with torch.no_grad():
                    x_pred = model.forwardRegressionAdapter(
                        x, channel_idx
                    )
                result_batches.append(torch.cat([
                    x.detach().cpu(),                       # (B, 4, T)
                    y.detach().cpu().unsqueeze(1),          # (B, 1, T)
                    x_pred.detach().cpu().unsqueeze(1),     # (B, 1, T)
                ], dim=1))
            result = torch.cat(result_batches, dim=0)       # (N, 5, T)
            result = torch.cat([
                result,
                data.denormalize(result[:, 3, :]).unsqueeze(1),
                data.denormalize(result[:, 4, :]).unsqueeze(1),
            ], dim=1).detach().cpu()                        # (N, 7, T)

            true_min = result[:, 5].min(dim=1).values.numpy()
            true_max = result[:, 5].max(dim=1).values.numpy()
            pred_min = result[:, 6].min(dim=1).values.numpy()
            pred_max = result[:, 6].max(dim=1).values.numpy()

            cond_ne1_array = cond_ne1_mask.to_numpy()
            if not true_filled:
                base_out["TrueMinBP"] = true_min[cond_ne1_array]
                base_out["TrueMaxBP"] = true_max[cond_ne1_array]
                true_filled = True

            tag = _weight_tag(w_min, w_max)
            base_out[f"PredMinBP_{tag}"] = pred_min[cond_ne1_array]
            base_out[f"PredMaxBP_{tag}"] = pred_max[cond_ne1_array]

            if device == "cuda":
                torch.cuda.empty_cache()

        out_rows.append(base_out)

    if not out_rows:
        print("No subjects processed; nothing saved.")
        return

    out_df = pd.concat(out_rows, axis=0).reset_index(drop=True)
    out_dir = Path("data/presentation/ResultFintune")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / "profile.csv"
    out_df.to_csv(out_path, index=False)
    print(f"Saved aggregated profile to {out_path} (rows={len(out_df)})")


# Ensure the working directory is the repo root so relative paths resolve
os.chdir(Path(__file__).resolve().parents[1])
main()


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

px.defaults.template = "plotly_white"
px.defaults.width = 1250
px.defaults.height = 600

def mae(y_true, y_pred) -> float:
    """Nan-tolerant MAE."""
    diff = np.abs(np.asarray(y_true) - np.asarray(y_pred))
    return float(np.nanmean(diff))

output_dir = Path("data/presentation/ResultFintune")
output_dir.mkdir(parents=True, exist_ok=True)

data_path = output_dir / "profile.csv"
profile = pd.read_csv(data_path)


In [ ]:
# Detect available parameter sets (tags)
pred_min_cols = [c for c in profile.columns if c.startswith("PredMinBP_")]
pred_max_cols = [c for c in profile.columns if c.startswith("PredMaxBP_")]
tags_min = [c.removeprefix("PredMinBP_") for c in pred_min_cols]
tags_max = [c.removeprefix("PredMaxBP_") for c in pred_max_cols]
assert tags_min, "No PredMinBP_* columns found"
assert tags_max, "No PredMaxBP_* columns found"
tags_min, tags_max

In [ ]:
# --- Strategy 1: global best parameter set (separate best for min vs. max) ---
def best_global(true_col: str, prefix: str, tags: list[str]):
    scores = {
        tag: mae(profile[true_col], profile[f"{prefix}{tag}"])
        for tag in tags
    }
    best_tag = min(scores, key=scores.get)
    return best_tag, scores

best_min_tag, min_scores = best_global("TrueMinBP", "PredMinBP_", tags_min)
best_max_tag, max_scores = best_global("TrueMaxBP", "PredMaxBP_", tags_max)
best_min_mae = min_scores[best_min_tag]
best_max_mae = max_scores[best_max_tag]

metrics_text = (
    f"Global best (min): {best_min_tag} -> MAE = {best_min_mae:.4f}\n"
    f"Global best (max): {best_max_tag} -> MAE = {best_max_mae:.4f}"
)
metrics_path = output_dir / "01GlobalBestParas.txt"
metrics_path.write_text(metrics_text + "\n")
print(f"Wrote global metrics to {metrics_path}")

# Per-sample errors for box plots
err_rows = []
for subj, df_sub in profile.groupby("subject"):
    err_rows.append(pd.DataFrame({
        "subject": subj,
        "metric": "diastolic",
        "error": np.abs(df_sub["TrueMinBP"] - df_sub[f"PredMinBP_{best_min_tag}"])
    }))
    err_rows.append(pd.DataFrame({
        "subject": subj,
        "metric": "systolic",
        "error": np.abs(df_sub["TrueMaxBP"] - df_sub[f"PredMaxBP_{best_max_tag}"])
    }))
sample_err_global = pd.concat(err_rows, ignore_index=True)

fig_box_global = px.box(
    sample_err_global, x="subject", y="error", color="metric",
    points="suspectedoutliers", title="Per-sample error (global best tags)",
    labels={"error": "|Pred-True|", "subject": "Subject"}
)

fig_box_global.update_traces(marker=dict(size=3), whiskerwidth=0)
fig_box_global.update_yaxes(type="log", range=[0, 2])
fig_box_global.write_image(output_dir / "01GlobalBestParas.png")


In [ ]:
# --- Strategy 2: per-subject best parameter set (separate for min vs. max) ---
def best_tags_per_subject(true_col: str, prefix: str, tags: list[str]):
    out = {}
    for subj, df_sub in profile.groupby("subject"):
        scores = {
            tag: mae(df_sub[true_col], df_sub[f"{prefix}{tag}"])
            for tag in tags
        }
        out[subj] = min(scores, key=scores.get)
    return out

best_min_by_subj = best_tags_per_subject("TrueMinBP", "PredMinBP_", tags_min)
best_max_by_subj = best_tags_per_subject("TrueMaxBP", "PredMaxBP_", tags_max)

# Build columns with per-subject best predictions
profile = profile.copy()
profile["PredMinBP_best_subject"] = np.nan
profile["PredMaxBP_best_subject"] = np.nan

for subj, tag in best_min_by_subj.items():
    mask = profile["subject"] == subj
    profile.loc[mask, "PredMinBP_best_subject"] = profile.loc[mask, f"PredMinBP_{tag}"]
for subj, tag in best_max_by_subj.items():
    mask = profile["subject"] == subj
    profile.loc[mask, "PredMaxBP_best_subject"] = profile.loc[mask, f"PredMaxBP_{tag}"]

overall_min_mae = mae(profile["TrueMinBP"], profile["PredMinBP_best_subject"])
overall_max_mae = mae(profile["TrueMaxBP"], profile["PredMaxBP_best_subject"])
metrics_text = (
    f"Per-subject best (min): MAE = {overall_min_mae:.4f}\n"
    f"Per-subject best (max): MAE = {overall_max_mae:.4f}"
)
metrics_path = output_dir / "02SubjectBestParas.txt"
metrics_path.write_text(metrics_text + "\n")
print(f"Wrote subject-level metrics to {metrics_path}")

# Per-sample errors for box plots (subject-specific best)
err_rows = []
for subj, df_sub in profile.groupby("subject"):
    err_rows.append(pd.DataFrame({
        "subject": subj,
        "metric": "diastolic",
        "error": np.abs(df_sub["TrueMinBP"] - df_sub["PredMinBP_best_subject"]),
    }))
    err_rows.append(pd.DataFrame({
        "subject": subj,
        "metric": "systolic",
        "error": np.abs(df_sub["TrueMaxBP"] - df_sub["PredMaxBP_best_subject"]),
    }))
sample_err_subject = pd.concat(err_rows, ignore_index=True)

fig_box_subj = px.box(
    sample_err_subject, x="subject", y="error", color="metric",
    points="suspectedoutliers", title="Per-sample error (subject-specific best tags)",
    labels={"error": "|Pred-True|", "subject": "Subject"}
)

fig_box_subj.update_traces(marker=dict(size=3), whiskerwidth=0)
fig_box_subj.update_yaxes(type="log", range=[0, 2])
fig_box_subj.write_image(output_dir / "02SubjectBestParas.png")

# Quick look at which tag won per subject (optional table)
pd.DataFrame({
    "subject": list(best_min_by_subj.keys()),
    "best_min_tag": list(best_min_by_subj.values()),
    "best_max_tag": [best_max_by_subj[s] for s in best_min_by_subj.keys()],
}).sort_values("subject")

In [ ]:
# Optional: remove extreme errors (possible bad ground truth) and recompute
pred_min_col = "PredMinBP_best_subject"
pred_max_col = "PredMaxBP_best_subject"

df_err = profile.copy()
df_err["err_diastolic"] = np.abs(df_err["TrueMinBP"] - df_err[pred_min_col])
df_err["err_systolic"] = np.abs(df_err["TrueMaxBP"] - df_err[pred_max_col])

def filter_outliers_iqr(df: pd.DataFrame, cols: list[str], k: float = 1.5, max_pct: float = 0.99):
    """IQR + percentile cap: keep rows where each col <= min(q3 + k*IQR, pct(max_pct))."""
    mask = pd.Series(True, index=df.index)
    thresholds: dict[str, float] = {}
    for col in cols:
        s = df[col].dropna()
        if s.empty:
            thresholds[col] = np.nan
            continue
        q1, q3 = s.quantile([0.25, 0.75])
        iqr = q3 - q1
        upper_iqr = q3 + k * iqr
        upper_pct = s.quantile(max_pct)
        upper = float(min(upper_iqr, upper_pct))
        thresholds[col] = upper
        mask &= df[col] <= upper
    return df[mask].copy(), thresholds, mask

filtered, thresholds, keep_mask = filter_outliers_iqr(
    df_err, ["err_diastolic", "err_systolic"], k=1.5, max_pct=0.99
)

summary_lines = [
    f"Thresholds (abs error): {thresholds}",
    (
        f"Kept {len(filtered)} of {len(df_err)} samples ("
        f"{keep_mask.mean()*100:.1f}%); removed {len(df_err) - len(filtered)}"
    ),
]

filtered_min_mae = mae(filtered["TrueMinBP"], filtered[pred_min_col])
filtered_max_mae = mae(filtered["TrueMaxBP"], filtered[pred_max_col])
summary_lines.append(
    "Filtered overall MAE (diastolic, systolic): "
    f"({filtered_min_mae:.4f}, {filtered_max_mae:.4f})"
)

metrics_path = output_dir / "03SubjectBestParasThresholds.txt"
metrics_path.write_text("\n".join(summary_lines) + "\n")
print(f"Wrote filtered metrics to {metrics_path}")

# Box plot on filtered data (box + suspected outliers on same line)
err_long = pd.concat([
    filtered[["subject", "err_diastolic"]].rename(columns={"err_diastolic": "error"}).assign(metric="diastolic"),
    filtered[["subject", "err_systolic"]].rename(columns={"err_systolic": "error"}).assign(metric="systolic"),
])
fig_box_filtered = px.box(
    err_long, x="subject", y="error", color="metric",
    points="suspectedoutliers",
    title="Per-sample error after outlier removal (subject-specific best)",
    labels={"error": "|Pred-True|", "subject": "Subject"}
)
fig_box_filtered.update_traces(marker=dict(size=3), whiskerwidth=0)
fig_box_filtered.update_yaxes(type="log", range=[0, 2])
fig_box_filtered.write_image(output_dir / "03SubjectBestParasThresholds.png")